# MD simulation of LJ liquid

**`By You-Liang Zhu`**

## Introduction

This tutorial is designed to provide an introduction to MD simulations with PYGAMD. It does however assume that you have a machine with PYGAMD correctly installed.

In this tutorial, we would like to make a system consisting of LJ liquids.

**Download [running files](https://github.com/youliangzhu/pygamd-v1/tree/main/examples/Case2-CGMD/LJ-liquid)**.

In order to run this MD simulation with PYGAMD, two files should be prepared:
*  `mst`: the file that describes all the information (including position, velocity, topology, etc.) of the particles in the system, more about [MST data format](https://pygamd-v1.readthedocs.io/en/latest/system-data-format.html); the mst configuration file could be generated by `molgen` plugin
*  `gala`: The file that describes the settings for the PYGAMD simulation engine

## Prepare topology and coordinate files

For this tutorial, you will build a system containing two kinds of molecules (shown as below) for simulation.
In order to build the two molecules, you need to write a python script of [`Molgen`](https://pygamd-v1.readthedocs.io/en/latest/molgen.html) that will run on a terminal of Linux to call the [`Molgen`](https://pygamd-v1.readthedocs.io/en/latest/molgen.html) plugin of PYGAMD.
The script "polymer.molg" to build a polymer-solvent system is shown as below in blue

In [ ]:
import molgen

mol=molgen.Molecule(1)
mol.setParticleTypes("a")

gen=molgen.Generators(20.0, 20.0, 20.0)
gen.addMolecule(mol,6000)
gen.setMinimumDistance(1.0);
gen.outPutMST("lj")

## Prepare PYGAMD DPD gala input files

Next component needed is the input files that define the program settings for MD run. For this system, firstly, we will perform an equilibrium run on the system at a high temperature of $k_BT$ = 1.0.
The JUGA file "lj_liquid.juga" is shown as below in blue:

In [ ]:
import pygamd

mst = pygamd.snapshot.read("lj.mst")
app = pygamd.application.dynamics(info=mst, dt=0.001)

fn = pygamd.force.nonbonded(info=mst, rcut=3.0, func='lj')
fn.setParams(type_i="a", type_j="a", param=[1.0, 1.0, 1.0, 3.0])
app.add(fn)

inn = pygamd.integration.nvt(info=mst, group='all', method="nh", tau=1.0, temperature=1.0)
app.add(inn)

dd = pygamd.dump.data(info=mst, group='all', file='data.log', period=100)
app.add(dd)

dm = pygamd.dump.mst(info=mst, group='all', file='p.mst', period=10000)
app.add(dm)

app.run(20000)